<a href="https://colab.research.google.com/github/statlib/learn-duckdb/blob/main/notebooks/name_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --pre duckdb doublemetaphone

In [2]:
import duckdb as dd
import duckdb.typing as T
from doublemetaphone import doublemetaphone

In [3]:
def dm(x: str, primary: bool=True) -> str:
    x = doublemetaphone(x)
    return x[0] if primary else x[1]

In [5]:
dd.create_function(
    'dm',
    dm,
    [T.VARCHAR, T.BOOLEAN],
    T.VARCHAR,
    type='native'
)

In [9]:
dd.sql(
    """
    DROP MACRO IF EXISTS name_dist;

    CREATE MACRO name_dist(a, b) AS (
        round(
            jaro_winkler_similarity(a, b),
            4
        )
    );

    DROP MACRO IF EXISTS sound_dist;

    CREATE MACRO sound_dist(a, b) AS (
        round(
            (
                jaro_winkler_similarity(dm(a, True), dm(b, True)) +
                jaro_winkler_similarity(dm(a, False), dm(b, False))
            ) / 2,
            4
        )
    );
    """
)

In [10]:
dd.sql(
    """
    WITH cte AS (
        SELECT
            *
        FROM (
            VALUES
                ('Jack Jackson'),
                ('Jack Johnson'),
                ('Jim Johnson')
        ) names(name)
    ),
    comb AS (
        SELECT
            c1.name name1,
            c2.name name2,
            name_dist(name1, name2) dist1
        FROM
            cte c1
        LEFT JOIN
            cte c2
        ON
            c1.name <> c2.name
        WHERE
            dist1 >= .8
    )
    SELECT
        *,
        sound_dist(name1, name2) dist2
    FROM
        comb
    WHERE
        dist2 >= .8
    """
)

┌──────────────┬──────────────┬────────┬────────┐
│    name1     │    name2     │ dist1  │ dist2  │
│   varchar    │   varchar    │ double │ double │
├──────────────┼──────────────┼────────┼────────┤
│ Jack Johnson │ Jack Jackson │ 0.8778 │ 0.8756 │
│ Jack Jackson │ Jack Johnson │ 0.8778 │ 0.8756 │
│ Jim Johnson  │ Jack Johnson │ 0.8705 │    0.9 │
│ Jack Johnson │ Jim Johnson  │ 0.8705 │    0.9 │
└──────────────┴──────────────┴────────┴────────┘